### temporary testing for accuracy

In [ ]:
#  imports
import numpy as np
import librosa
import numpy as np
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt

from chromagram import fetch_chromagram
from DTW import dtw_table, dtw, display_dtw_path, similarity_score

#### generate chromagrams for train files from dataset

In [ ]:
# fetch file pairs


gen_path = "../dataset3/archive/musicnet/custom_generated_musicnet/"
og_path = "../dataset3/archive/musicnet/musicnet/train_data/"

# where train files chromagrams will be saved, to be loaded and used during inference
chroma_path = "../chromagrams/30sec_sr10000_numpy_files/"

file_pairs = []
no_pairs = []
multiple_pairs = []

og_files = os.listdir(og_path)


for file in tqdm(os.listdir(gen_path)):
    file_num = file.split('_')[0]
    topair = [f for f in og_files if file_num in f]
    if len(topair)==0:
        no_pairs.append(file)
        continue
    elif len(topair)>1:
        multiple_pairs.append([file,topair])
        continue
    topair = topair[0]
    file_pairs.append([topair, file])
file_pairs = np.array(file_pairs)

In [ ]:
# save chromagrams

og_files = file_pairs[:,0]
for ogf in tqdm(og_files):
    # saving 30 seconds chromagrams of all train files with sampling rate 10k
    # change accordingly
    chromagram = fetch_chromagram(og_path+ogf, sr=10000, end_time=30)
    np.save(chroma_path+ogf[:-3]+'npy', chromagram)

#### inference (computing closest matches for test files)

In [ ]:
def method1(chromagram1, chromagram2):
    # dtw from MIR with cost without normalization
    D = dtw_table(chromagram1.T, chromagram2.T)
    dtw_path = dtw(chromagram1.T, chromagram1.T, D)

    score = similarity_score(D, dtw_path)
    
    return score

def method2(chromagram1, chromagram2):
    # librosa dtw with normalized similarity
    return 0

In [ ]:
chroma_files = os.listdir(chroma_path)

# each element in final_maps contains [original file, generated midi/test file, closest match file]
final_maps = []

for og_file, gen_file in tqdm(file_pairs):
    # load test file chromagram
    chromagram1 = fetch_chromagram(gen_path+gen_file, sr=10000, end_time=30)
    scores = []
    for chroma_file in tqdm(chroma_files):
        # fetch each train file chromagram
        chromagram2 = np.load(chroma_path+chroma_file)
        
        score = method1(chromagram1, chromagram2)
        
        scores.append(score)
        
        
    matched_file = chroma_files[np.argmin(scores)]    # argmin in case of cost, argmax in case of similarity
    
    # each element in final_maps contains [original file, generated midi/test file, closest match file]
    final_maps.append([og_file, gen_file, matched_file])
    
    
    break    # test for one test file print final_maps to check, remove break later

##### accuracy

In [ ]:
accuracy = 0
for i in final_maps:
    ground_truth = i[0][:-3]
    closest_match = i[2][:-3]
    accuracy += ground_truth==closest_match
accuracy = accuracy/len(final_maps)
accuracy*100

In [ ]:
accuracy